In [118]:
from bs4 import BeautifulSoup
import requests as req
from time import sleep
import pandas as pd
import os
import time
from datetime import datetime as dt
from tqdm import tqdm
#from tqdm import tqdm


# configure pandas
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"
    )
}

## Rank Data
____

In [121]:
url = "https://www.teamrankings.com/nfl/stats/"
page = req.get(url, headers=headers).text
soup = BeautifulSoup(page, "html.parser")
urls = [a["href"] for a in soup.find(attrs={"class":"column large-2"}).find_all("a") if a["href"] != "#"]
            

In [122]:
urls

['/nfl/stat/points-per-game',
 '/nfl/stat/average-scoring-margin',
 '/nfl/stat/yards-per-point',
 '/nfl/stat/yards-per-point-margin',
 '/nfl/stat/points-per-play',
 '/nfl/stat/points-per-play-margin',
 '/nfl/stat/touchdowns-per-game',
 '/nfl/stat/red-zone-scoring-attempts-per-game',
 '/nfl/stat/red-zone-scores-per-game',
 '/nfl/stat/red-zone-scoring-pct',
 '/nfl/stat/extra-point-attempts-per-game',
 '/nfl/stat/extra-points-made-per-game',
 '/nfl/stat/two-point-conversion-attempts-per-game',
 '/nfl/stat/two-point-conversions-per-game',
 '/nfl/stat/points-per-field-goal-attempt',
 '/nfl/stat/extra-point-conversion-pct',
 '/nfl/stat/two-point-conversion-pct',
 '/nfl/stat/offensive-touchdowns-per-game',
 '/nfl/stat/defensive-touchdowns-per-game',
 '/nfl/stat/special-teams-touchdowns-per-game',
 '/nfl/stat/offensive-points-per-game',
 '/nfl/stat/defensive-points-per-game',
 '/nfl/stat/special-teams-points-per-game',
 '/nfl/stat/offensive-point-share-pct',
 '/nfl/stat/1st-quarter-points-per-

In [115]:
date = "2022-10-26"
week = 7

dates_w_week = {
    "2022-10-26":7,
    "2022-10-19":6,
    "2022-10-12":5,
    "2022-10-05":4,
    "2022-09-28":3,
}

url = "https://www.teamrankings.com/nfl/stats/"
page = req.get(url, headers=headers).text
soup = BeautifulSoup(page, "html.parser")
pbar = tqdm(["/nfl/stat/points-per-game"])
pd_list = []
for _date, week in dates_w_week.items():
    for end_point in pbar:
        time.sleep(.25)
        url = f"https://www.teamrankings.com{end_point}?date={_date}"
        page = req.get(url, headers=headers).text
        soup = BeautifulSoup(page, "html.parser")
        table = soup.find("table")
        df = pd.read_html(str(table))[0].replace("--",0)
        df["asof"] = date
        df["week "] = week
        df["url"] = end_point
        file_name = end_point.replace('/', '_').replace("-","_") + "_week" + str(week)
        pbar.set_description(end_point)
        pd_list.append(df)
    
full_file_path = f"/rank_data_week_{week}.csv"
pd.concat(pd_list).to_csv(f"{full_file_path}.csv")
print(df.head())


/nfl/stat/points-per-game:  50%|█████     | 1/2 [00:00<00:00,  1.09it/s]


ConnectionError: HTTPSConnectionPool(host='www.teamrankings.comopponent-points-per-game', port=443): Max retries exceeded with url: /?date=2022-10-26 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f63148a4ef0>: Failed to establish a new connection: [Errno -2] Name or service not known',))

### Spread and Game Data
___

In [ ]:
url = f"https://www.sportsoddshistory.com/nfl-game-season/?y=2022"
page = req.get(url, headers=headers).text
soup = BeautifulSoup(page, "html.parser")
tables = soup.find_all("table", attrs={"class":"soh1"})
pd_list = []
for i, table in enumerate(tables[2:]):
    df = pd.read_html(str(table))
    df = df[0]
    df.columns = df.iloc[-2]
    df = df.iloc[:-2,: ]
    df["week"] = i + 1
    pd_list.append(df)
df = pd.concat(pd_list)
print(df.head())
df.to_csv("data/nfl_weekly_betting_summary.csv")
df.to_csv("data/ml_data/nfl_weekly_betting_summary.csv")

In [109]:
df_games = pd.read_csv("nfl_weekly_betting_summary.csv")
df_ranks = pd.read_csv("rank_data_week_8.csv")

In [110]:
df_ranks["_stat"] = df_ranks.url.str.split("/", expand=True)[3]
df_ranks["Stat"] = df_ranks._stat.str.split("-").str.join(" ").str.title()
df_ranks["Stat"] = df_ranks.Stat.str.replace("Per Game", "Pg").str.replace("Opponent", "Opp")\
.str.replace("Offensive", "Off")\
.str.replace("Defensive", "Def")\
.str.replace("Touchdowns", "Td")\
.str.replace("Rushing", "Run")\
.str.replace("Passing", "Pass")\
.str.replace("Interceptions", "Int")

In [111]:
df_ranks.head()

,Unnamed: 0,Rank,Team,2022,Last 3,Last 1,Home,Away,2021,asof,url,_stat,Stat
0,0,1,Kansas City,31.9,31.3,44.0,25.7,36.5,29.4,2022-11-02,/nfl/stat/points-per-game,points-per-game,Points Pg
1,1,2,Buffalo,29.0,29.7,27.0,35.3,24.3,29.8,2022-11-02,/nfl/stat/points-per-game,points-per-game,Points Pg
2,2,3,Philadelphia,28.0,27.0,35.0,28.5,27.3,25.5,2022-11-02,/nfl/stat/points-per-game,points-per-game,Points Pg
3,3,4,Seattle,26.3,27.7,27.0,21.5,31.0,23.2,2022-11-02,/nfl/stat/points-per-game,points-per-game,Points Pg
4,4,5,Baltimore,26.0,23.3,27.0,25.0,27.0,22.8,2022-11-02,/nfl/stat/points-per-game,points-per-game,Points Pg


In [112]:
df_games.head()

,Unnamed: 0,Day,Date,Time (ET),Unnamed: 4,Favorite,Score,Spread,Unnamed: 8,Underdog,Over/Under,Notes,week
0,0,Thu,"Sep 8, 2022",8:20,NaN,Buffalo Bills,W 31-10,W -2.5,@,Los Angeles Rams,U 52,NaN,1
1,1,Sun,"Sep 11, 2022",1:00,NaN,New Orleans Saints,W 27-26,L -5.5,@,Atlanta Falcons,O 43.5,NaN,1
2,2,Sun,"Sep 11, 2022",1:00,@,Carolina Panthers,L 24-26,L -1.5,NaN,Cleveland Browns,O 42,NaN,1
3,3,Sun,"Sep 11, 2022",1:00,NaN,San Francisco 49ers,L 10-19,L -6,@,Chicago Bears,U 38,NaN,1
4,4,Sun,"Sep 11, 2022",1:00,@,Cincinnati Bengals,L 20-23 (OT),L -7,NaN,Pittsburgh Steelers,U 44,NaN,1


In [92]:
df_rank["map_key"] = df.week

In [100]:
df_ranks[df_ranks._stat.eq("opponent-points-per-game") & df_ranks.Team.eq("Kansas City")].head()

,Unnamed: 0,Rank,Team,2022,Last 3,Last 1,Home,Away,2021,asof,url,_stat,Stat
3254,22,23,Kansas City,24.6,25.3,23.0,25.7,23.8,22.4,2022-11-02,/nfl/stat/opponent-points-per-game,opponent-points-per-game,Opp Points Pg


In [103]:
df_games[df_games.Favorite.str.startswith("Kansas City")].head()

,Unnamed: 0,Day,Date,Time (ET),Unnamed: 4,Favorite,Score,Spread,Unnamed: 8,Underdog,Over/Under,Notes,week
10,10,Sun,"Sep 11, 2022",4:25,NaN,Kansas City Chiefs,W 44-21,W -6.5,@,Arizona Cardinals,O 54,NaN,1
16,0,Thu,"Sep 15, 2022",8:15,@,Kansas City Chiefs,W 27-24,L -4,NaN,Los Angeles Chargers,U 54,NaN,2
35,3,Sun,"Sep 25, 2022",1:00,NaN,Kansas City Chiefs,L 17-20,L -5.5,@,Indianapolis Colts,U 51,NaN,3
79,15,Mon,"Oct 10, 2022",8:15,@,Kansas City Chiefs,W 30-29,L -7,NaN,Las Vegas Raiders,O 51.5,NaN,5
105,11,Sun,"Oct 23, 2022",4:25,NaN,Kansas City Chiefs,W 44-23,W -1,@,San Francisco 49ers,O 49,NaN,7


In [82]:
df_games["fav_pts"] = df_games.Score.str.split(" ", expand=True)[1].str.split("-", expand=True)[0]
df_games["dog_pts"] = df_games.Score.str.split(" ", expand=True)[1].str.split("-", expand=True)[1]
df_fav = df_games[["Date", "Time (ET)", "Favorite", "Underdog", "Score", "week", "fav_pts"]].rename(columns={"Favorite":"Team", 
                                                                                                 "fav_pts":"team_pts_scored", 
                                                                                                 "Underdog":"Opp",
                                                                                                })

In [83]:
df_opp = df_games[["Date", "Time (ET)", "Favorite", "Underdog", "Score", "week", "dog_pts"]].rename(columns={"Underdog":"Team", 
                                                                                                 "dog_pts":"team_pts_scored",
                                                                                                 "Favorite":"Opp"
                                                                                                })
df_games = pd.concat([df_fav, df_opp])

In [90]:
df_ranks.head()

,Unnamed: 0,Rank,Team,2022,Last 3,Last 1,Home,Away,2021,asof,url
0,0,1,Kansas City,31.9,31.3,44.0,25.7,36.5,29.4,2022-11-02,/nfl/stat/points-per-game
1,1,2,Buffalo,29.0,29.7,27.0,35.3,24.3,29.8,2022-11-02,/nfl/stat/points-per-game
2,2,3,Philadelphia,28.0,27.0,35.0,28.5,27.3,25.5,2022-11-02,/nfl/stat/points-per-game
3,3,4,Seattle,26.3,27.7,27.0,21.5,31.0,23.2,2022-11-02,/nfl/stat/points-per-game
4,4,5,Baltimore,26.0,23.3,27.0,25.0,27.0,22.8,2022-11-02,/nfl/stat/points-per-game


In [89]:
df_games.head()

,Unnamed: 0,Day,Date,Time (ET),Unnamed: 4,Favorite,Score,Spread,Unnamed: 8,Underdog,Over/Under,Notes,week
0,0,Thu,"Sep 8, 2022",8:20,NaN,Buffalo Bills,W 31-10,W -2.5,@,Los Angeles Rams,U 52,NaN,1
1,1,Sun,"Sep 11, 2022",1:00,NaN,New Orleans Saints,W 27-26,L -5.5,@,Atlanta Falcons,O 43.5,NaN,1
2,2,Sun,"Sep 11, 2022",1:00,@,Carolina Panthers,L 24-26,L -1.5,NaN,Cleveland Browns,O 42,NaN,1
3,3,Sun,"Sep 11, 2022",1:00,NaN,San Francisco 49ers,L 10-19,L -6,@,Chicago Bears,U 38,NaN,1
4,4,Sun,"Sep 11, 2022",1:00,@,Cincinnati Bengals,L 20-23 (OT),L -7,NaN,Pittsburgh Steelers,U 44,NaN,1
